In [ ]:
#We first import all the needed libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pyarrow as pa


In [4]:
import pandas as pd

# Load CSV as object first to avoid conversion errors

df = pd.read_csv(r"E:\BA-Project\raw_data\foundational_industry_data_2017.csv", dtype=str, low_memory=False)

# Check the dataframe

print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644881 entries, 0 to 644880
Data columns (total 62 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Unnamed: 0                 644881 non-null  object
 1   registryID                 644881 non-null  object
 2   fuelTypeStd                62492 non-null   object
 3   eisUnitID                  157962 non-null  object
 4   SCC                        52464 non-null   object
 5   unitType                   165297 non-null  object
 6   unitDescription            154952 non-null  object
 7   designCapacity             48388 non-null   object
 8   designCapacityUOM          48972 non-null   object
 9   fuelType                   89729 non-null   object
 10  eisProcessID               157962 non-null  object
 11  processDescription         110270 non-null  object
 12  energyMJq0                 46227 non-null   object
 13  energyMJq2                 46227 non-null   

In [ ]:
#Our data has a lot of unneeded columns so we drop them to optimize memory usage and a lot of rows have missing values in important columns so we drop those rows as well

# We sort the dataframe by energyMJq0 column to bring NaN values to the end

df = df.sort_values(by="energyMJq0", na_position="last")

df.head()

,Unnamed: 0,registryID,fuelTypeStd,eisUnitID,SCC,unitType,unitDescription,designCapacity,designCapacityUOM,fuelType,...,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4,geoID,legislativeDistrictNumber
16446,16446,110041038098.0,diesel,133195713.0,20100102.0,unknown,emergency standby diesel generator,NaN,NaN,distillate_oil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,060855045044,0617
53798,53798,110070522664.0,naturalGas,132898513.0,20100202.0,unknown,emergency standby generator set,NaN,NaN,natural_gas,...,93.2,103.2,111.7,102.3,109.271996,123.78,132.672,121.312004,060014371011,0615
103807,103807,110000426913.0,naturalGas,102738313.0,20200254.0,unknown,internal comb engine - industrial,NaN,NaN,natural_gas,...,78.2,70.6,67.7,78.6,102.7,95.296,91.808,115.644,270159604003,2701
48047,48047,110042051089.0,naturalGas,130348713.0,39990013.0,THERMAL RECUPERATIVE OXIDIZER,thermal recuperative oxidizer,NaN,NaN,natural_gas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104260,104260,110000427538.0,lpgHGL,102505413.0,10201002.0,other,metal parts wash system,NaN,NaN,LPG,...,54.9,52.7,52.9,52.4,69.012,64.264,65.248,62.004,271359703001,2707


In [ ]:
#We drop the unneeded columns from the dataframe

columns_to_drop = [
    "ghgrpID", "ghgrpIDAdditional", "eisFacilityIDAdditional",
    "eisUnitID", "eisProcessID", "geoID", "hucCode8", "epaRegionCode",
    "sensitiveInd", "smallBusInd", "envJusticeCode",
    "processDescription", "designCapacity",
    "naicsCodeAdditional", "Unnamed: 0", "unitType"
]

df = df.drop(columns=columns_to_drop, errors="ignore")
df.head()

,registryID,fuelTypeStd,SCC,unitDescription,designCapacityUOM,fuelType,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,...,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4,legislativeDistrictNumber
16446,110041038098.0,diesel,20100102.0,emergency standby diesel generator,NaN,distillate_oil,0.0003033125,3.1328475559488473,5.8198555867867166,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0617
53798,110070522664.0,naturalGas,20100202.0,emergency standby generator set,NaN,natural_gas,0.000324729,1.18416011934361,7.8604098545454555,NaN,...,83.288,93.2,103.2,111.7,102.3,109.271996,123.78,132.672,121.312004,0615
103807,110000426913.0,naturalGas,20200254.0,internal comb engine - industrial,NaN,natural_gas,0.00033213111186671207,76.07430222088836,76.52212113136588,5.449401564093846e-09,...,41.556,78.2,70.6,67.7,78.6,102.7,95.296,91.808,115.644,2701
48047,110042051089.0,naturalGas,39990013.0,thermal recuperative oxidizer,NaN,natural_gas,0.0003326012181818182,2797.2450957864867,2874.676154704937,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104260,110000427538.0,lpgHGL,10201002.0,metal parts wash system,NaN,LPG,0.00034626638799163987,76.38243713338267,77.648,NaN,...,42.796,54.9,52.7,52.9,52.4,69.012,64.264,65.248,62.004,2707


In [7]:
df.describe(include='all')

,registryID,fuelTypeStd,SCC,unitDescription,designCapacityUOM,fuelType,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,...,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4,legislativeDistrictNumber
count,644881,62492,52464,154952,48972,89729,46227,46227,46227,29760,...,324049,324049,324049,324049,324049,324049,324049,324049,324049,494542
unique,502165,10,711,108891,27,102,33591,35763,36901,21661,...,78,75,76,77,75,71,72,72,71,439
top,110020057070.0,naturalGas,10200602.0,boiler,MW,natural_gas,2886480.0,4329720.0,4329720.0,9.070000000000001e-06,...,57.415997,64.2,61.0,60.3,63.1,69.687996,66.096,65.2,68.784,0804
freq,602,40392,5366,578,46050,36923,169,113,71,106,...,35216,35216,35216,35216,35216,35216,35216,35216,35216,9421


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 644881 entries, 16446 to 644880
Data columns (total 46 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   registryID                 644881 non-null  object
 1   fuelTypeStd                62492 non-null   object
 2   SCC                        52464 non-null   object
 3   unitDescription            154952 non-null  object
 4   designCapacityUOM          48972 non-null   object
 5   fuelType                   89729 non-null   object
 6   energyMJq0                 46227 non-null   object
 7   energyMJq2                 46227 non-null   object
 8   energyMJq3                 46227 non-null   object
 9   throughputTonneQ0          29760 non-null   object
 10  throughputTonneQ2          29760 non-null   object
 11  throughputTonneQ3          29760 non-null   object
 12  ghgsTonneCO2eQ0            47624 non-null   object
 13  ghgsTonneCO2eQ2            47624 non-null   o

In [ ]:
#these columns have too many nulls and are not useful for our analysis so we drop them as well

drop_cols = [
    "unitName", "energyMJ", "ghgsTonneCO2e", "unitDescription",
    "designCapacityUOM", "SCC", "fuelType",
    "eisFacilityID", "legislativeDistrictNumber"
]

df = df.drop(columns=drop_cols, errors='ignore')
df.head()

,registryID,fuelTypeStd,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,throughputTonneQ2,throughputTonneQ3,ghgsTonneCO2eQ0,ghgsTonneCO2eQ2,...,weeklyOpHoursLow_q3,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4
16446,110041038098.0,diesel,0.0003033125,3.1328475559488473,5.8198555867867166,NaN,NaN,NaN,2.133500125e-08,0.00022036449708544192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53798,110070522664.0,naturalGas,0.000324729,1.18416011934361,7.8604098545454555,NaN,NaN,NaN,1.634685786e-08,5.9610620407757334e-05,...,90.728,83.288,93.2,103.2,111.7,102.3,109.271996,123.78,132.672,121.312004
103807,110000426913.0,naturalGas,0.00033213111186671207,76.07430222088836,76.52212113136588,5.449401564093846e-09,5.449401564093846e-09,5.449401564093846e-09,1.851187e-06,1.851187e-06,...,43.591995,41.556,78.2,70.6,67.7,78.6,102.7,95.296,91.808,115.644
48047,110042051089.0,naturalGas,0.0003326012181818182,2797.2450957864867,2874.676154704937,NaN,NaN,NaN,1.6743145323272728e-08,0.14081331812189174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104260,110000427538.0,lpgHGL,0.00034626638799163987,76.38243713338267,77.648,NaN,NaN,NaN,5.426127499999999e-06,5.426127499999999e-06,...,40.552002,42.796,54.9,52.7,52.9,52.4,69.012,64.264,65.248,62.004


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 644881 entries, 16446 to 644880
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   registryID            644881 non-null  object
 1   fuelTypeStd           62492 non-null   object
 2   energyMJq0            46227 non-null   object
 3   energyMJq2            46227 non-null   object
 4   energyMJq3            46227 non-null   object
 5   throughputTonneQ0     29760 non-null   object
 6   throughputTonneQ2     29760 non-null   object
 7   throughputTonneQ3     29760 non-null   object
 8   ghgsTonneCO2eQ0       47624 non-null   object
 9   ghgsTonneCO2eQ2       47624 non-null   object
 10  ghgsTonneCO2eQ3       47624 non-null   object
 11  unitTypeStd           165298 non-null  object
 12  energyEstimateSource  160447 non-null  object
 13  ghgsEstimateSource    154025 non-null  object
 14  siteTypeName          555741 non-null  object
 15  name              

In [ ]:
# We change the data types of numeric columns to numeric, coercing errors to NaN


numeric_cols = [
    "energyMJq0","energyMJq2","energyMJq3",
    "throughputTonneQ0","throughputTonneQ2","throughputTonneQ3",
    "ghgsTonneCO2eQ0","ghgsTonneCO2eQ2","ghgsTonneCO2eQ3",
    "latitude","longitude","postalCode","countyFIPS", "weeklyOpHoursLow_q3", "weeklyOpHoursLow_q4", "weeklyOpHoursLow_q1", "weeklyOpHours_q1",
    "weeklyOpHours_q2", "weeklyOpHours_q3", "weeklyOpHoursLow_q2", "weeklyOpHours_q4", "weeklyOpHoursHigh_q1", "weeklyOpHoursHigh_q2", "weeklyOpHoursHigh_q3", "weeklyOpHoursHigh_q4"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
# we fill missing numeric values with the median of each column
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())


In [13]:
df.head()

,registryID,fuelTypeStd,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,throughputTonneQ2,throughputTonneQ3,ghgsTonneCO2eQ0,ghgsTonneCO2eQ2,...,weeklyOpHoursLow_q3,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4
16446,110041038098.0,diesel,0.000303,3.132848,5.819856,1.962375e+02,4.190191e+02,5.888033e+02,2.133500e-08,0.000220,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000
53798,110070522664.0,naturalGas,0.000325,1.184160,7.860410,1.962375e+02,4.190191e+02,5.888033e+02,1.634686e-08,0.000060,...,90.728000,83.288,93.2,103.2,111.7,102.3,109.271996,123.780,132.672,121.312004
103807,110000426913.0,naturalGas,0.000332,76.074302,76.522121,5.449402e-09,5.449402e-09,5.449402e-09,1.851187e-06,0.000002,...,43.591995,41.556,78.2,70.6,67.7,78.6,102.700000,95.296,91.808,115.644000
48047,110042051089.0,naturalGas,0.000333,2797.245096,2874.676155,1.962375e+02,4.190191e+02,5.888033e+02,1.674315e-08,0.140813,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000
104260,110000427538.0,lpgHGL,0.000346,76.382437,77.648000,1.962375e+02,4.190191e+02,5.888033e+02,5.426127e-06,0.000005,...,40.552002,42.796,54.9,52.7,52.9,52.4,69.012000,64.264,65.248,62.004000


In [14]:
df.describe()

C:\Users\hagar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\hagar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\hagar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,throughputTonneQ2,throughputTonneQ3,ghgsTonneCO2eQ0,ghgsTonneCO2eQ2,ghgsTonneCO2eQ3,postalCode,...,weeklyOpHoursLow_q3,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4
count,6.448810e+05,6.448810e+05,6.448810e+05,6.448810e+05,6.448810e+05,6.448810e+05,6.448810e+05,6.448810e+05,6.448810e+05,644881.000000,...,644881.000000,644881.000000,644881.000000,644881.000000,644881.000000,644881.00000,644881.000000,644881.000000,644881.000000,644881.000000
mean,1.203879e+07,2.549619e+07,3.449722e+07,inf,inf,inf,5.611206e+04,1.255164e+05,1.520513e+05,58886.153490,...,54.625615,54.597400,68.571681,72.523024,71.953840,69.79502,89.291573,92.758095,88.293072,87.294473
std,4.393048e+08,5.766452e+08,6.783979e+08,NaN,NaN,NaN,3.893675e+07,5.068719e+07,5.246601e+07,25933.913152,...,17.417245,17.977024,18.633256,17.667283,18.852338,19.10455,21.041570,22.328094,24.335616,23.728726
min,6.050106e-05,4.420846e-03,4.420850e-03,8.098214e-11,8.098214e-11,8.098214e-11,0.000000e+00,0.000000e+00,0.000000e+00,601.000000,...,3.859999,3.859999,29.400000,30.000000,33.500000,31.30000,40.491997,40.592000,40.488003,40.588000
25%,1.889332e+06,7.399885e+06,9.309980e+06,1.962375e+02,4.190191e+02,5.888033e+02,4.763399e+01,1.946055e+02,2.456882e+02,40391.000000,...,50.992000,51.544000,64.200000,69.100000,67.700000,64.30000,87.704000,91.192000,82.444000,81.332000
50%,1.889332e+06,7.399885e+06,9.309980e+06,1.962375e+02,4.190191e+02,5.888033e+02,4.763399e+01,1.946055e+02,2.456882e+02,60617.000000,...,50.992000,51.544000,64.200000,69.100000,67.700000,64.30000,87.704000,91.192000,82.444000,81.332000
75%,1.889332e+06,7.399885e+06,9.309980e+06,1.962375e+02,4.190191e+02,5.888033e+02,4.763399e+01,1.946055e+02,2.456882e+02,80022.000000,...,50.992000,51.876000,64.500000,69.100000,67.700000,64.30000,87.704000,91.192000,82.444000,81.332000
max,7.122873e+10,7.506945e+10,7.591244e+10,inf,inf,inf,3.122250e+10,3.122250e+10,3.122250e+10,99950.000000,...,165.528000,165.824000,168.000000,166.000000,166.900000,167.00000,168.000000,168.000000,168.000000,168.000000


In [17]:
df.duplicated().sum()

np.int64(70585)

In [18]:
df[df.duplicated()]

,registryID,fuelTypeStd,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,throughputTonneQ2,throughputTonneQ3,ghgsTonneCO2eQ0,ghgsTonneCO2eQ2,...,weeklyOpHoursLow_q3,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4
104262,110000427538.0,lpgHGL,3.462664e-04,7.638244e+01,77.648,196.237541,419.01906,588.803324,0.000005,0.000005,...,40.552002,42.796,54.9,52.7,52.9,52.4,69.012000,64.264,65.248,62.004000
104266,110000427538.0,lpgHGL,3.462664e-04,7.638244e+01,77.648,196.237541,419.01906,588.803324,0.000005,0.000005,...,40.552002,42.796,54.9,52.7,52.9,52.4,69.012000,64.264,65.248,62.004000
104226,110000427538.0,lpgHGL,1.405059e-03,7.638244e+01,77.648,196.237541,419.01906,588.803324,0.000005,0.000005,...,40.552002,42.796,54.9,52.7,52.9,52.4,69.012000,64.264,65.248,62.004000
104103,110000427191.0,naturalGas,5.528000e-03,1.082430e+05,108243.000,196.237541,419.01906,588.803324,0.002619,0.002619,...,50.576004,51.876,64.5,61.4,78.8,75.2,80.768005,75.316,107.024,98.523994
107501,110007601997.0,lpgHGL,6.834433e-03,4.853000e+01,48.530,196.237541,419.01906,588.803324,0.000004,0.000004,...,45.664000,45.564,54.2,54.5,53.7,53.6,61.648003,63.320,61.736,61.635998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165293,110071017155.0,NaN,1.889332e+06,7.399885e+06,9309980.430,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000
165294,110071017155.0,NaN,1.889332e+06,7.399885e+06,9309980.430,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000
165295,110071017155.0,NaN,1.889332e+06,7.399885e+06,9309980.430,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000
165296,110071017155.0,NaN,1.889332e+06,7.399885e+06,9309980.430,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000


In [ ]:
#We remove duplicate rows from the dataframe

df = df.drop_duplicates()

In [ ]:
# We convert the fuelTypeStd column's "None" strings to actual NaN values

df[df["fuelTypeStd"]=="None"] = np.nan;
df.tail()

,registryID,fuelTypeStd,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,throughputTonneQ2,throughputTonneQ3,ghgsTonneCO2eQ0,ghgsTonneCO2eQ2,...,weeklyOpHoursLow_q3,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4
644876,110071648305.0,NaN,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644877,110071648309.0,NaN,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644878,110071648456.0,NaN,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644879,110071648563.0,NaN,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644880,110071648755.0,NaN,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332


In [ ]:
#We fill missing values in fuelTypeStd column with the mode of the column
md = df["fuelTypeStd"].mode();
print(md)

0    naturalGas
Name: fuelTypeStd, dtype: object


In [28]:

df["fuelTypeStd"].fillna(inplace= True, value=md[0])
df.tail()

,registryID,fuelTypeStd,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,throughputTonneQ2,throughputTonneQ3,ghgsTonneCO2eQ0,ghgsTonneCO2eQ2,...,weeklyOpHoursLow_q3,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4
644876,110071648305.0,naturalGas,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644877,110071648309.0,naturalGas,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644878,110071648456.0,naturalGas,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644879,110071648563.0,naturalGas,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332
644880,110071648755.0,naturalGas,1.889332e+06,7.399885e+06,9309980.43,196.237541,419.01906,588.803324,47.633988,194.605451,...,50.992,51.544,64.2,69.1,67.7,64.3,87.704,91.192,82.444,81.332


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574296 entries, 16446 to 644880
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   registryID            574296 non-null  object 
 1   fuelTypeStd           574296 non-null  object 
 2   energyMJq0            574296 non-null  float64
 3   energyMJq2            574296 non-null  float64
 4   energyMJq3            574296 non-null  float64
 5   throughputTonneQ0     574296 non-null  float64
 6   throughputTonneQ2     574296 non-null  float64
 7   throughputTonneQ3     574296 non-null  float64
 8   ghgsTonneCO2eQ0       574296 non-null  float64
 9   ghgsTonneCO2eQ2       574296 non-null  float64
 10  ghgsTonneCO2eQ3       574296 non-null  float64
 11  unitTypeStd           94713 non-null   object 
 12  energyEstimateSource  92161 non-null   object 
 13  ghgsEstimateSource    86556 non-null   object 
 14  siteTypeName          489597 non-null  object 
 15  n

In [ ]:
#we now change the data types of categorical columns to 'string' to optimize memory usage

cat = ["energyEstimateSource", "ghgsEstimateSource", "siteTypeName", "name", "locationAddress", "cityName", "countyName",
       "stateCode", "naicsCode", "unitTypeStd", "fuelTypeStd", "registryID"]

for col in cat:
    df[col] = df[col].astype('string')    



df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574296 entries, 16446 to 644880
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   registryID            574296 non-null  string 
 1   fuelTypeStd           574296 non-null  string 
 2   energyMJq0            574296 non-null  float64
 3   energyMJq2            574296 non-null  float64
 4   energyMJq3            574296 non-null  float64
 5   throughputTonneQ0     574296 non-null  float64
 6   throughputTonneQ2     574296 non-null  float64
 7   throughputTonneQ3     574296 non-null  float64
 8   ghgsTonneCO2eQ0       574296 non-null  float64
 9   ghgsTonneCO2eQ2       574296 non-null  float64
 10  ghgsTonneCO2eQ3       574296 non-null  float64
 11  unitTypeStd           94713 non-null   string 
 12  energyEstimateSource  92161 non-null   string 
 13  ghgsEstimateSource    86556 non-null   string 
 14  siteTypeName          489597 non-null  string 
 15  n

In [ ]:
#We validate the latitude and longitude values


df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df = df[(df['latitude'].between(-90, 90)) & (df['longitude'].between(-180, 180))]
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 574296 entries, 16446 to 644880
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   registryID            574296 non-null  object 
 1   fuelTypeStd           574296 non-null  object 
 2   energyMJq0            574296 non-null  float64
 3   energyMJq2            574296 non-null  float64
 4   energyMJq3            574296 non-null  float64
 5   throughputTonneQ0     574296 non-null  float64
 6   throughputTonneQ2     574296 non-null  float64
 7   throughputTonneQ3     574296 non-null  float64
 8   ghgsTonneCO2eQ0       574296 non-null  float64
 9   ghgsTonneCO2eQ2       574296 non-null  float64
 10  ghgsTonneCO2eQ3       574296 non-null  float64
 11  unitTypeStd           94713 non-null   object 
 12  energyEstimateSource  92161 non-null   object 
 13  ghgsEstimateSource    86556 non-null   object 
 14  siteTypeName          489597 non-null  object 
 15  n

In [ ]:
#Finally this is what the cleaned dataframe looks like
df.head()

,registryID,fuelTypeStd,energyMJq0,energyMJq2,energyMJq3,throughputTonneQ0,throughputTonneQ2,throughputTonneQ3,ghgsTonneCO2eQ0,ghgsTonneCO2eQ2,...,weeklyOpHoursLow_q3,weeklyOpHoursLow_q4,weeklyOpHours_q1,weeklyOpHours_q2,weeklyOpHours_q3,weeklyOpHours_q4,weeklyOpHoursHigh_q1,weeklyOpHoursHigh_q2,weeklyOpHoursHigh_q3,weeklyOpHoursHigh_q4
16446,110041038098.0,diesel,0.000303,3.132848,5.819856,1.962375e+02,4.190191e+02,5.888033e+02,2.133500e-08,0.000220,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000
53798,110070522664.0,naturalGas,0.000325,1.184160,7.860410,1.962375e+02,4.190191e+02,5.888033e+02,1.634686e-08,0.000060,...,90.728000,83.288,93.2,103.2,111.7,102.3,109.271996,123.780,132.672,121.312004
103807,110000426913.0,naturalGas,0.000332,76.074302,76.522121,5.449402e-09,5.449402e-09,5.449402e-09,1.851187e-06,0.000002,...,43.591995,41.556,78.2,70.6,67.7,78.6,102.700000,95.296,91.808,115.644000
48047,110042051089.0,naturalGas,0.000333,2797.245096,2874.676155,1.962375e+02,4.190191e+02,5.888033e+02,1.674315e-08,0.140813,...,50.992000,51.544,64.2,69.1,67.7,64.3,87.704000,91.192,82.444,81.332000
104260,110000427538.0,lpgHGL,0.000346,76.382437,77.648000,1.962375e+02,4.190191e+02,5.888033e+02,5.426127e-06,0.000005,...,40.552002,42.796,54.9,52.7,52.9,52.4,69.012000,64.264,65.248,62.004000


In [ ]:
#We save the cleaned dataframe to a parquet file to optimize storage and retrieval

table = pa.Table.from_pandas(df, preserve_index=False)
pq.write_table(table, "cleaned_data.parquet", compression="snappy")